<h3>Librerías, paquetes y funciones usadas en el cuaderno</h3>

In [7]:
import cv2
import math
import numpy as np
from ultralytics import YOLO
from sklearn.cluster import KMeans

<h3>Funciones para la obtención de la ROI de cada jugador y su posterior clasificación de equipo por colores</h3>

In [10]:
# Función para obtener el ROI del área central de la bbox
def get_central_roi(frame, x1, y1, x2, y2):
    # Coordenadas de la bbox
    width = x2 - x1
    height = y2 - y1

    # Definir subárea como proporción de la bbox
    sub_x1 = int(x1 + width * 0.3)  # Dejar un 30% de margen a los lados
    sub_x2 = int(x2 - width * 0.3)  # Dejar un 30% de margen a los lados
    sub_y1 = int(y1 + height * 0.3)  # Subárea empieza al 30% de la altura
    sub_y2 = int(y1 + height * 0.6)  # Subárea llega al 60% de la altura

    # Recortar y devolver el ROI
    roi = frame[sub_y1:sub_y2, sub_x1:sub_x2]
    return roi

# Función para obtener los colores de los equipos
def classifyTeam(player_roi):
    
    # Convertir a HSV y preparar los datos para KMeans (matriz de (n_pixels, 3))
    roi_hsv = cv2.cvtColor(player_roi, cv2.COLOR_BGR2HSV)
    pixels = roi_hsv.reshape(-1, 3)

    # Aplicar KMeans para obtener los colores dominantes
    k = 2
    kmeans = KMeans(n_clusters=k, random_state=0).fit(pixels)
    dominant_colors = kmeans.cluster_centers_
    labels, counts = np.unique(kmeans.labels_, return_counts=True)

    # Obtener el color más dominante
    dominant_cluster = labels[np.argmax(counts)]
    dominant_color = dominant_colors[dominant_cluster]

    # Si no se han clasificado los colores de los equipos, asignar el color a uno de ellos
    if np.array_equal(team_colors['A'], np.array([0,0,0])):
        team_colors['A'] = dominant_color
        return 'A'
    
    # Si ya se ha clasificado el color de un equipo, asignar el color al otro
    elif np.array_equal(team_colors['B'], np.array([0,0,0])):
        # Si el color es muy similar al del equipo A, evita asignarlo a B
        if np.linalg.norm(dominant_color - team_colors['A']) < 100:
            return 'A'
        team_colors['B'] = dominant_color
        return 'B'
    
    # Si ya se han clasificado ambos colores, asignar el color al equipo más cercano
    else:
        if np.linalg.norm(dominant_color - team_colors['A']) < np.linalg.norm(dominant_color - team_colors['B']):
            return 'A'
        else:
            return 'B'

# Función para obtener el color BGR de un equipo a partir de su nombre
def getTeamBGRColor(team):
    if team not in team_colors:
        return [0, 0, 0]
    bgr_color = team_colors[team].astype(np.uint8)
    bgr_color = cv2.cvtColor(bgr_color[np.newaxis, np.newaxis, :], cv2.COLOR_HSV2BGR)[0, 0]
    return bgr_color.tolist()

def drawBBox(frame, x1, y1, x2, y2, team, label):
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), getTeamBGRColor(team), 2)
    cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 200, 100), 2)

def drawPosition(frame, position, position_label):
    cv2.ellipse(frame, (int(position[0]), int(position[1])), (20, 10), 0, 0, 360, (0, 0, 255), -1)
    cv2.putText(frame, position_label, (int(position[0]) - 50, int(position[1]) + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)


<h3>Detección de jugadores y posicionamiento</h3>

In [ ]:
# Cargar el modelo YOLO entrenado
model = YOLO("./yolo_models/bpdv1.pt")

video_path = "./assets/videos/dal-lac1.mp4"
# Cargar el video de entrada
cap = cv2.VideoCapture(video_path)

# Obtener detalles del video
fourcc = cv2.VideoWriter_fourcc(*"avc1")  # Codec para el video de salida
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Configurar el video de salida
output_path = "video_resultado.mp4"
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Inicializar colores de los equipos
team_colors = {'A': np.array([0,0,0]), 'B': np.array([0,0,0])}

# Procesar cada frame del video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Realizar detección en el frame
    results = model(frame)

    # Análisis de las detecciones
    for result in results[0].boxes.data.tolist():  # Obtener los resultados como lista
        x1, y1, x2, y2, conf, cls = result  # Coordenadas, confianza y clase
        cls = int(cls)

        # Filtrar solo por las clases deseadas
        if model.names[cls] in ['player', 'referee'] and conf > 0.35:

            # Calcular la posición como el punto medio del borde inferior de la bbox
            position = (int((x1 + x2) / 2), y2)
            position_label = f"x:{int(position[0])} y:{int(position[1])}"

            label = f"{model.names[cls]} {conf:.2f}"

            # Si la clase es 'player', clasificar el equipo y dibujar la bbox en el frame
            if model.names[cls] == 'player':

                player_roi = get_central_roi(frame, int(x1), int(y1), int(x2), int(y2))

                team = classifyTeam(player_roi)

                drawBBox(frame, x1, y1, x2, y2, team, label)

            # Si la clase es 'referee', dibujar la bbox en el frame
            if model.names[cls] == 'referee':
                drawBBox(frame, x1, y1, x2, y2, 'referee', label)

            # Dibujar la posición en el frame
            drawPosition(frame, position, position_label)


    # Mostrar el frame procesado en pantalla
    cv2.imshow('Resultados', frame)

    # Escribir el frame procesado en el video de salida
    out.write(frame)

    # Esperar 1ms para salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video procesado guardado en {output_path}")

---

---

<h3>Testing</h3>

Tracking usando BoT-SORT (default YOLO)

In [ ]:
# Cargar el modelo YOLO entrenado
model = YOLO("./yolo_models/bpdv1.pt")

# Cargar el video de entrada
video_path = "./assets/videos/okc-nyk.mp4"
cap = cv2.VideoCapture(video_path)

# Obtener detalles del video
fourcc = cv2.VideoWriter_fourcc(*"avc1")  # Codec para el video de salida
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Configurar el video de salida
output_path = "video_tracking_bot-sort.mp4"
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

team_colors = {'A': np.array([0,0,0]), 'B': np.array([0,0,0])}
player_team_map = {}


# Procesar cada frame del video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Realizar detección en el frame
    results = model.track(frame, persist=True)

    # Dibujar las detecciones en el frame
    for result in results:  # Obtener los resultados como lista
        boxes = result.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            
            # Confianza
            confidence = math.ceil((box.conf[0]*100))/100
            print("Confianza --->",confidence)

            # Clase
            cls = int(box.cls[0])
            print("Clase -->", model.names[cls])

            #Etiqueta de seguimiento
            if box.id is not None:
                track_id = str(int(box.id[0].tolist()))
            else:
                track_id = ''
            
            print(track_id)
            


            # Filtrar solo por las clases deseadas
            if model.names[cls] in ['player', 'referee'] and confidence > 0.35:
                label = f"{model.names[cls]} {conf:.2f}"
                player_position = (int((x1 + x2) / 2), y2)
                player_position_label = f"x:{int(player_position[0])} y:{int(player_position[1])}"
                bgr_color = np.array([0, 0, 0])
                team = ''

                # Si la clase es 'player', clasificar el equipo y dibujar la bbox en el frame
                if model.names[cls] == 'player':

                    player_roi = get_central_roi(frame, int(x1), int(y1), int(x2), int(y2))

                    team = classifyTeam(player_roi)

                    drawBBox(frame, x1, y1, x2, y2, team, label)

                # Si la clase es 'referee', dibujar la bbox en el frame
                if model.names[cls] == 'referee':
                    drawBBox(frame, x1, y1, x2, y2, 'referee', label)

                # Dibujar la posición en el frame
                drawPosition(frame, position, position_label)

                cv2.putText(frame, track_id + ' ' + model.names[cls] , [x1, y1 - 30], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Mostrar el frame procesado en pantalla
    cv2.imshow('Detecciones', frame)

    # Escribir el frame procesado en el video de salida
    out.write(frame)

    # Esperar 1ms para salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video procesado guardado en {output_path}")



Tracking usando DeepSORT Realtime

In [ ]:
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2

# Configuración de YOLO
model = YOLO("./yolo_models/bpdv1.pt")

# Configuración de DeepSORT
tracker = DeepSort(max_age=15, nn_budget=20)

# Configurar el video de entrada
video_path = "./assets/videos/okc-nyk.mp4"
cap = cv2.VideoCapture(video_path)
out = cv2.VideoWriter("video_tracking_deepsort.mp4", cv2.VideoWriter_fourcc(*"avc1"), 30, 
                      (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detecciones de YOLO
    results = model(frame)
    detections = []
    for result in results[0].boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = result[:6]
        if model.names[int(cls)] not in ["player", "referee"]:
            continue
        width = x2 - x1
        height = y2 - y1
        detections.append(([x1, y1, width, height], conf, int(cls)))

    print(detections)
    # Seguimiento con DeepSORT
    if len(detections) > 0:
        tracks = tracker.update_tracks(detections, frame=frame)

        # Visualización de resultados
        for track in tracks:
            if not track.is_confirmed():
                continue
            track_id = track.track_id
            l, t, w, h = track.to_ltwh()
            cv2.rectangle(frame, (int(l), int(t)), (int(l + w), int(t + h)), (0, 255, 0), 2)
            label = f"ID: {track_id}"
            cv2.putText(frame, label, (int(l), int(t) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Video", frame)
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
